In [1]:
# ! pip install mlxtend
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

sales_fact_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")
product_ori_df = pd.read_csv("./hw1_data/product.csv")
customer_ori_df = pd.read_csv("./hw1_data/customer.csv")
transaction_ori_df = pd.concat([sales_fact_1998_ori_df, sales_fact_dec_1998_ori_df]).reset_index() # concatenate two datasets

time: 262 ms (started: 2022-01-20 08:20:22 +00:00)


# Q1

In [2]:
transaction_df = transaction_ori_df.copy().drop(['promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_df = pd.concat([transaction_df, pd.get_dummies(transaction_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_df = transaction_df.drop_duplicates()
transaction_df = transaction_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
display(transaction_df)

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
732     534         10                          0               0   
        1013        10                          0               0   
        1325        10                          0               0   
        1425        10                          0               0   
        1550        10                          0               0   
...                                           ...             ...   
1095    9559        10                          0               0   
        9696        11                          0               0   
        9725        10                          0               0   
        9743        10                          0               0   
        10095       10                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
732     534         10                                 0   
        1013        10                                 0   
        1325        10                                 0   
        1425        10                                 0   
        1550        10                                 0   
...                                                  ...   
1095    9559        10                                 0   
        9696        11                                 0   
        9725        10                                 0   
        9743        10                                 0   
        10095       10                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
732     534         10                            0   
        1013        10                            0   
        1325        10                            0   
        1425        10                            0   
        1550        10                            0   
...                                             ...   
1095    9559        10                            0   
        9696        11                            0   
        9725        10                            0   
        9743        10                            0   
        10095       10                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
732     534         10                                0                   0   
        1013        10                                0                   0   
        1325        10                                0                   0   
        1425        10                                0                   0   
        1550        10                                0                   0   
...                                                 ...                 ...   
1095    9559        10                                0                   0   
        9696        11                                0                   0   
        9725        10                                0                   0   
        9743        10                                0                   0   
        10095       10                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
732     534         10                           0   
        1013        10                           0   
        1325        10                           0   
        1425        10                           0   
        1550        10                           0   
...                                            ...   
1095    9559        10                           0   
        9696        11                           0   
        9725        10                           0   
        9743 

time: 32.6 s (started: 2022-01-20 08:20:22 +00:00)


In [3]:
# an_col = [col for col in transaction_df.columns if (set(transaction_df[col].unique().tolist()) != {0,1})]
# duplicate_ind = []
# for ind in transaction_df.loc[(transaction_df==2).any(axis=1),an_col].index:
#     cond = np.logical_and(np.logical_and(transaction_ori_df['time_id']==ind[0], transaction_ori_df['customer_id']==ind[1]), transaction_ori_df['store_id']==ind[2])
#     tmp = transaction_ori_df.loc[cond , 'product_id'].duplicated()
#     duplicate_ind.append(tmp.loc[tmp==1].index.values[0])
    
# duplicate_ind

time: 358 µs (started: 2022-01-20 08:20:54 +00:00)


In [4]:
transac_apriori_freq_itemset = apriori(transaction_df, min_support=0.0001, use_colnames=True, low_memory=True) # use apriori to find frequent itemset
transac_apriori_rules = association_rules(transac_apriori_freq_itemset, metric='confidence', min_threshold=0.9) # use frequent itemset to find rules
transac_apriori_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('apriori_product.csv')
display(transac_apriori_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_apriori_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
1084,0.004148,(Moms Roasted Chicken)
746,0.004095,(Great English Muffins)
1389,0.003989,(Super Chunky Peanut Butter)
748,0.003963,(Great Pumpernickel Bread)
1117,0.003936,(Nationeel Dried Apples)
...,...,...
9085,0.000106,"(Toucan Canned Mixed Fruit, Even Better 2% Milk)"
9086,0.000106,"(Walrus Light Wine, Even Better 2% Milk)"
9088,0.000106,"(Hermanos Fancy Plums, Even Better Blueberry Y..."
9090,0.000106,"(Even Better Blueberry Yogurt, Imagine Lime Po..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
381,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.0,320.771186,0.000184,inf
702,"(Just Right Canned Yams, Hilltop 200 MG Acetom...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.0,337.955357,0.000158,inf
325,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
598,"(High Quality Scissors, Plato French Roast Cof...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
550,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.0,300.404762,0.000158,inf
...,...,...,...,...,...,...,...,...,...
1427,"(Dual City Lox, Plato Grape Jam, Cutting Edge ...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
759,"(High Quality Paper Towels, Tell Tale Lemons)",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1118,"(Booker Low Fat Cottage Cheese, Big Time Popsi...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1116,"(Booker Low Fat Cottage Cheese, Consolidated M...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf


time: 3.55 s (started: 2022-01-20 08:20:54 +00:00)


# Q2

In [5]:
transac_fp_freq_itemset = fpgrowth(transaction_df, min_support=0.0001, use_colnames=True) # use fp-growth to find frequent_itemset
transac_fp_rules = association_rules(transac_fp_freq_itemset, metric='confidence', min_threshold=0.9) # find rules
transac_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_product.csv')
display(transac_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
383,0.004148,(Moms Roasted Chicken)
454,0.004095,(Great English Muffins)
595,0.003989,(Super Chunky Peanut Butter)
1509,0.003963,(Great Pumpernickel Bread)
1032,0.003936,(Carrington Ice Cream)
...,...,...
7334,0.000106,"(Genteel Extra Lean Hamburger, Best Choice Dri..."
7335,0.000106,"(Best Choice Dried Apples, Pleasant Canned Tun..."
7336,0.000106,"(Nationeel Mini Donuts, Best Choice Dried Apples)"
7337,0.000106,"(Ebony Broccoli, Best Choice Dried Apples)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
805,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.0,320.771186,0.000184,inf
309,"(Just Right Canned Yams, Hilltop 200 MG Acetom...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.0,337.955357,0.000158,inf
1249,"(High Quality Scissors, Plato French Roast Cof...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
1534,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.0,312.818182,0.000158,inf
1190,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.0,300.404762,0.000158,inf
...,...,...,...,...,...,...,...,...,...
1050,"(Booker Low Fat Cottage Cheese, Big Time Popsi...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
1048,"(Booker Low Fat Cottage Cheese, Consolidated M...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
453,"(High Quality Paper Towels, Tell Tale Lemons)",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf
448,"(Dual City Lox, Plato Grape Jam, Cutting Edge ...",(Great English Muffins),0.000106,0.004095,0.000106,1.0,244.200000,0.000105,inf


time: 1.35 s (started: 2022-01-20 08:20:58 +00:00)


# Q3

In [6]:
cus_attr = ["state_province", "yearly_income", "gender", "total_children", "num_children_at_home", "education", "occupation", "houseowner", "num_cars_owned"]
customer_attr_df = customer_ori_df.loc[:, cus_attr] # we choose the wanted columns
discret_cus_df = customer_attr_df.select_dtypes(include='int').apply(lambda x:pd.cut(x, x.max())) # eqivalent frequency cut the numerical attributes
customer_cat_df = pd.concat([customer_attr_df.select_dtypes(exclude='int'), discret_cus_df], axis=1) # concatenate categorical attributes
customer_oh_cat_df = pd.get_dummies(customer_cat_df)
display(customer_oh_cat_df)

,state_province_BC,state_province_CA,state_province_DF,state_province_Guerrero,state_province_Jalisco,state_province_Mexico,state_province_OR,state_province_Oaxaca,state_province_Sinaloa,state_province_Veracruz,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10276,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
10277,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
10278,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
10279,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0


time: 27.6 ms (started: 2022-01-20 08:20:59 +00:00)


In [7]:
cus_fp_freq_itemset = fpgrowth(customer_oh_cat_df, min_support=0.05, use_colnames=True)
cus_fp_rules = association_rules(cus_fp_freq_itemset, metric='confidence', min_threshold=0.9)
cus_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_customer.csv')
display(cus_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        cus_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
8,0.773466,"(num_children_at_home_(-0.005, 1.0])"
0,0.601790,(houseowner_Y)
9,0.504231,(gender_M)
1,0.495769,(gender_F)
33,0.427877,"(houseowner_Y, num_children_at_home_(-0.005, 1..."
...,...,...
180,0.050190,"(gender_M, num_children_at_home_(-0.005, 1.0],..."
531,0.050092,"(education_Partial High School, occupation_Man..."
545,0.050092,"(yearly_income_$10K - $30K, num_cars_owned_(-0..."
547,0.050092,"(num_children_at_home_(-0.005, 1.0], yearly_in..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.297928,0.773466,0.297928,1.000000,1.292882,0.067491,inf
15,"(houseowner_Y, total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.178290,0.773466,0.178290,1.000000,1.292882,0.040389,inf
5,"(gender_M, total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.154654,0.773466,0.154654,1.000000,1.292882,0.035034,inf
16,"(total_children_(-0.005, 1.0], gender_F)","(num_children_at_home_(-0.005, 1.0])",0.143274,0.773466,0.143274,1.000000,1.292882,0.032456,inf
20,"(state_province_CA, total_children_(-0.005, 1.0])","(num_children_at_home_(-0.005, 1.0])",0.124696,0.773466,0.124696,1.000000,1.292882,0.028248,inf
...,...,...,...,...,...,...,...,...,...
2,"(education_High School Degree, occupation_Skil...",(yearly_income_$30K - $50K),0.056707,0.323607,0.051260,0.903945,2.793345,0.032909,7.041738
38,"(gender_M, education_Partial High School, num_...",(yearly_income_$10K - $30K),0.065752,0.216127,0.059333,0.902367,4.175173,0.045122,8.028761
0,"(yearly_income_$30K - $50K, houseowner_N, gend...","(num_children_at_home_(-0.005, 1.0])",0.066725,0.773466,0.060208,0.902332,1.166610,0.008599,2.319442
91,"(education_High School Degree, occupation_Manual)",(yearly_income_$30K - $50K),0.113899,0.323607,0.102714,0.901793,2.786696,0.065855,6.887449


time: 174 ms (started: 2022-01-20 08:20:59 +00:00)


# Q4

In [8]:
transaction_df = transaction_ori_df.copy().drop(['time_id', 'store_id', 'promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1) # we only need product_id, customer_id
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
customer_cat_cid_df = pd.concat([customer_cat_df, customer_ori_df['customer_id']], axis=1)
transac_cus_df = pd.merge(transaction_df, customer_cat_cid_df, on='customer_id')
transac_cus_oh_df = pd.get_dummies(transac_cus_df.drop(['customer_id'], axis=1))
display(transac_cus_oh_df)

,product_id_ADJ Rosy Sunglasses,product_id_Akron City Map,product_id_Akron Eyeglass Screwdriver,product_id_American Beef Bologna,product_id_American Chicken Hot Dogs,product_id_American Cole Slaw,product_id_American Corned Beef,product_id_American Foot-Long Hot Dogs,product_id_American Low Fat Bologna,product_id_American Low Fat Cole Slaw,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182878,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182879,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182880,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
182881,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


time: 27.6 s (started: 2022-01-20 08:21:00 +00:00)


In [9]:
transac_cus_fp_freq_itemset = fpgrowth(transac_cus_oh_df, min_support=0.0001, max_len=2, use_colnames=True) # limit length of itemsets to 2
transac_cus_fp_rules = association_rules(transac_cus_fp_freq_itemset, metric='confidence', min_threshold=0.001) # can't find rules when min_threshold larger than 0.001
filter_ind = transac_cus_fp_rules['consequents'].apply(lambda x:x.isdisjoint(customer_oh_cat_df.keys().tolist())==True).tolist() # choose the consequents consist only of product
transac_cus_filtered_fp_rules = transac_cus_fp_rules.iloc[filter_ind,:] # choose the consequents consist only of product
transac_cus_filtered_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False).to_csv('fp_quanti_rules.csv')
display(transac_cus_fp_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_cus_filtered_fp_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
0,0.772439,"(num_children_at_home_(-0.005, 1.0])"
1,0.604534,(houseowner_Y)
2,0.510436,(gender_F)
167,0.489564,(gender_M)
1607,0.426688,"(houseowner_Y, num_children_at_home_(-0.005, 1..."
...,...,...
3826,0.000104,"(product_id_Carrington Lime Popsicles, total_c..."
18934,0.000104,"(product_id_Best Wheat Puffs, yearly_income_$7..."
18937,0.000104,"(product_id_Best Wheat Puffs, num_children_at_..."
18945,0.000104,"(product_id_Best Wheat Puffs, state_province_OR)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18752,"(num_children_at_home_(2.0, 3.0])",(product_id_Hermanos Peaches),0.070105,0.000760,0.000104,0.001482,1.949801,0.000051,1.000723
18044,"(num_children_at_home_(2.0, 3.0])",(product_id_Best Wheat Puffs),0.070105,0.000776,0.000104,0.001482,1.908608,0.000049,1.000707
2610,(state_province_Yucatan),(product_id_Better Noodle Soup),0.071160,0.000755,0.000104,0.001460,1.934804,0.000050,1.000706
4878,(state_province_Yucatan),(product_id_High Top New Potatos),0.071160,0.000771,0.000104,0.001460,1.893638,0.000049,1.000690
7946,(education_Partial College),(product_id_Nationeel Grape Fruit Roll),0.093344,0.000705,0.000131,0.001406,1.993131,0.000065,1.000702
...,...,...,...,...,...,...,...,...,...
31772,(yearly_income_$50K - $70K),(product_id_Tri-State Honey Dew),0.169387,0.000705,0.000170,0.001001,1.418704,0.000050,1.000296
36317,(yearly_income_$50K - $70K),(product_id_Ebony Corn on the Cob),0.169387,0.000705,0.000170,0.001001,1.418704,0.000050,1.000296
34493,(yearly_income_$50K - $70K),(product_id_Pleasant Canned Tuna in Water),0.169387,0.000716,0.000170,0.001001,1.397045,0.000048,1.000285
2564,(yearly_income_$50K - $70K),(product_id_Framton City Map),0.169387,0.000749,0.000170,0.001001,1.335860,0.000043,1.000252


time: 7.83 s (started: 2022-01-20 08:21:27 +00:00)


# Q5

In [10]:
transaction_before_dec_df = sales_fact_1998_ori_df.copy().drop(['time_id', 'store_id', 'promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1)  # we only need product_id, customer_id
transaction_dec_df = sales_fact_dec_1998_ori_df.copy().drop(['time_id', 'store_id', 'promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1)  # we only need product_id, customer_id
transaction_before_dec_df['product_id'] = transaction_before_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_dec_df['product_id'] = transaction_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
customer_cat_cid_df = pd.concat([customer_cat_df, customer_ori_df['customer_id']], axis=1)
transac_cus_before_dec_df = pd.merge(transaction_before_dec_df, customer_cat_cid_df, on='customer_id')
transac_cus_dec_df = pd.merge(transaction_dec_df, customer_cat_cid_df, on='customer_id')
transac_cus_before_dec_oh_df = pd.get_dummies(transac_cus_before_dec_df.drop(['customer_id'], axis=1))
transac_cus_dec_oh_df = pd.get_dummies(transac_cus_dec_df.drop(['customer_id'], axis=1))
display(transac_cus_before_dec_oh_df, transac_cus_dec_oh_df)

,product_id_ADJ Rosy Sunglasses,product_id_Akron City Map,product_id_Akron Eyeglass Screwdriver,product_id_American Beef Bologna,product_id_American Chicken Hot Dogs,product_id_American Cole Slaw,product_id_American Corned Beef,product_id_American Foot-Long Hot Dogs,product_id_American Low Fat Bologna,product_id_American Low Fat Cole Slaw,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164553,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
164554,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
164555,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
164556,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


,product_id_ADJ Rosy Sunglasses,product_id_Akron City Map,product_id_Akron Eyeglass Screwdriver,product_id_American Beef Bologna,product_id_American Chicken Hot Dogs,product_id_American Cole Slaw,product_id_American Corned Beef,product_id_American Foot-Long Hot Dogs,product_id_American Low Fat Bologna,product_id_American Low Fat Cole Slaw,...,"total_children_(4.0, 5.0]","num_children_at_home_(-0.005, 1.0]","num_children_at_home_(1.0, 2.0]","num_children_at_home_(2.0, 3.0]","num_children_at_home_(3.0, 4.0]","num_children_at_home_(4.0, 5.0]","num_cars_owned_(-0.004, 1.0]","num_cars_owned_(1.0, 2.0]","num_cars_owned_(2.0, 3.0]","num_cars_owned_(3.0, 4.0]"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18320,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
18321,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
18322,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0
18323,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,0


time: 27.1 s (started: 2022-01-20 08:21:35 +00:00)


In [11]:
transac_cus_before_dec_freq_itemset = fpgrowth(transac_cus_before_dec_oh_df, min_support=0.0001, max_len=2, use_colnames=True) # Inorder to simplify rules, limit length of itemset to 2
transac_cus_dec_freq_itemset = fpgrowth(transac_cus_dec_oh_df, min_support=0.0001, max_len=2, use_colnames=True) # Inorder to simplify rules, limit length of itemset to 2
transac_cus_before_dec_rules = association_rules(transac_cus_before_dec_freq_itemset, metric='confidence', min_threshold=0.001) # find rules 
transac_cus_dec_rules = association_rules(transac_cus_dec_freq_itemset, metric='confidence', min_threshold=0.001) # find rules
filter_ind_before_dec = transac_cus_before_dec_rules['consequents'].apply(lambda x:x.isdisjoint(customer_oh_cat_df.keys().tolist())==True).tolist() # choose the consequents consist only of product
filter_ind_dec = transac_cus_dec_rules['consequents'].apply(lambda x:x.isdisjoint(customer_oh_cat_df.keys().tolist())==True).tolist() # choose the consequents consist only of product
transac_cus_before_dec_filtered_rules = transac_cus_before_dec_rules.iloc[filter_ind_before_dec,:] # choose the consequents consist only of product
transac_cus_dec_filtered_rules = transac_cus_dec_rules.iloc[filter_ind_dec,:] # choose the consequents consist only of product
display(transac_cus_before_dec_freq_itemset.sort_values(by = 'support', ascending=False), 
        transac_cus_dec_freq_itemset.sort_values(by = 'support', ascending=False))
display(transac_cus_before_dec_filtered_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False), 
        transac_cus_dec_filtered_rules.sort_values(by = ['confidence', 'support', 'lift', 'leverage', 'antecedent support', 'consequent support', 'antecedents'], ascending=False))

,support,itemsets
0,0.772500,"(num_children_at_home_(-0.005, 1.0])"
1,0.604255,(houseowner_Y)
2,0.509680,(gender_F)
160,0.490320,(gender_M)
1607,0.426524,"(houseowner_Y, num_children_at_home_(-0.005, 1..."
...,...,...
22216,0.000103,"(product_id_Super Pepper, yearly_income_$50K -..."
22266,0.000103,"(state_province_Zacatecas, product_id_Better C..."
22290,0.000103,"(state_province_CA, product_id_Tri-State Limes)"
22308,0.000103,"(product_id_Blue Label Fancy Canned Clams, tot..."


,support,itemsets
13,0.771896,"(num_children_at_home_(-0.005, 1.0])"
0,0.607040,(houseowner_Y)
1,0.517217,(gender_F)
31,0.482783,(gender_M)
1607,0.428158,"(houseowner_Y, num_children_at_home_(-0.005, 1..."
...,...,...
30181,0.000109,"(product_id_Tell Tale Corn on the Cob, state_p..."
20013,0.000109,"(product_id_Carlson Buttermilk, total_children..."
9553,0.000109,"(product_id_Pleasant Turkey Noodle Soup, total..."
9554,0.000109,"(product_id_Pleasant Turkey Noodle Soup, state..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
17927,"(num_children_at_home_(2.0, 3.0])",(product_id_Hermanos Peaches),0.069641,0.000747,0.000109,0.001571,2.101366,0.000057,1.000825
26735,"(num_children_at_home_(2.0, 3.0])",(product_id_Best Wheat Puffs),0.069641,0.000784,0.000109,0.001571,2.003628,0.000055,1.000788
4749,(state_province_Yucatan),(product_id_High Top New Potatos),0.071610,0.000754,0.000109,0.001527,2.027109,0.000055,1.000775
7592,(education_Partial College),(product_id_Nationeel Grape Fruit Roll),0.093414,0.000717,0.000140,0.001496,2.086577,0.000073,1.000780
18986,(state_province_DF),(product_id_Bird Call 200 MG Acetominifen),0.069319,0.000705,0.000103,0.001490,2.114163,0.000054,1.000787
...,...,...,...,...,...,...,...,...,...
13771,(occupation_Manual),(product_id_Swell Canned Peaches),0.241927,0.000735,0.000243,0.001005,1.366440,0.000065,1.000270
18957,(occupation_Manual),(product_id_Hilltop Mint Mouthwash),0.241927,0.000778,0.000243,0.001005,1.291713,0.000055,1.000227
18773,(occupation_Manual),(product_id_Skinner Strawberry Drink),0.241927,0.000784,0.000243,0.001005,1.281699,0.000053,1.000221
14476,(yearly_income_$30K - $50K),(product_id_Nationeel Golden Raisins),0.327945,0.000802,0.000328,0.001001,1.247437,0.000065,1.000199


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
42490,(state_province_Jalisco),(product_id_Blue Medal Large Brown Eggs),0.009386,0.000709,0.000164,0.017442,24.586315,0.000157,1.017029
47062,(occupation_Clerical),(product_id_Portsmouth Chardonnay Wine),0.016207,0.000546,0.000218,0.013468,24.680135,0.000209,1.013099
20394,(occupation_Clerical),(product_id_Tell Tale Lettuce),0.016207,0.000982,0.000218,0.013468,13.711186,0.000202,1.012656
38799,(state_province_Jalisco),(product_id_Great Bagels),0.009386,0.000655,0.000109,0.011628,17.756783,0.000103,1.011102
39416,(state_province_Jalisco),(product_id_PigTail Apple Cinnamon Waffles),0.009386,0.000655,0.000109,0.011628,17.756783,0.000103,1.011102
...,...,...,...,...,...,...,...,...,...
43527,(yearly_income_$10K - $30K),(product_id_Booker Low Fat Sour Cream),0.215989,0.001037,0.000218,0.001011,0.974708,-0.000006,0.999974
44761,(yearly_income_$10K - $30K),(product_id_Fast Low Fat Cookies),0.215989,0.001037,0.000218,0.001011,0.974708,-0.000006,0.999974
46378,(yearly_income_$10K - $30K),(product_id_Best Choice Raspberry Fruit Roll),0.215989,0.001037,0.000218,0.001011,0.974708,-0.000006,0.999974
25294,(yearly_income_$10K - $30K),(product_id_Hilltop Childrens Cold Remedy),0.215989,0.001091,0.000218,0.001011,0.925973,-0.000017,0.999919


time: 9.32 s (started: 2022-01-20 08:22:02 +00:00)


In [12]:
common_rules = pd.merge(transac_cus_before_dec_filtered_rules, transac_cus_dec_filtered_rules, on=["antecedents", "consequents"], suffixes=('_before', '_dec'))
common_rules = common_rules.sort_values(by = ['confidence_before', 'confidence_dec', 
                                              'support_before', 'support_dec', 
                                              'lift_before', 'lift_dec', 
                                              'leverage_before', 'leverage_dec', 
                                              'antecedent support_before', 'antecedent support_dec', 
                                              'consequent support_before', 'consequent support_dec', 
                                              'antecedents'], ascending=False)
common_rules.to_csv('dec_common_rules.csv')
display(common_rules)

,antecedents,consequents,antecedent support_before,consequent support_before,support_before,confidence_before,lift_before,leverage_before,conviction_before,antecedent support_dec,consequent support_dec,support_dec,confidence_dec,lift_dec,leverage_dec,conviction_dec
10,(state_province_Yucatan),(product_id_Better Noodle Soup),0.071610,0.000735,0.000103,0.001443,1.961959,0.000051,1.000708,0.067121,0.000928,0.000109,0.001626,1.752750,0.000047,1.000699
55,(state_province_Zacatecas),(product_id_Better Canned Tuna in Oil),0.137477,0.000808,0.000182,0.001326,1.640735,0.000071,1.000519,0.127040,0.000709,0.000218,0.001718,2.422020,0.000128,1.001011
109,(education_Partial College),(product_id_Big Time Frozen Corn),0.093414,0.000699,0.000122,0.001301,1.861748,0.000056,1.000603,0.092715,0.000764,0.000164,0.001766,2.311233,0.000093,1.001004
58,(education_Partial College),(product_id_Consolidated 200 MG Ibuprofen),0.093414,0.000644,0.000122,0.001301,2.019820,0.000061,1.000658,0.092715,0.000546,0.000109,0.001177,2.157151,0.000059,1.000632
69,"(num_children_at_home_(1.0, 2.0])",(product_id_Booker Low Fat Sour Cream),0.095042,0.000632,0.000122,0.001279,2.023387,0.000061,1.000648,0.095061,0.001037,0.000109,0.001148,1.107318,0.000011,1.000111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,"(total_children_(1.0, 2.0])",(product_id_Black Tie Eyeglass Screwdriver),0.204682,0.000772,0.000207,0.001009,1.307966,0.000049,1.000238,0.212278,0.000982,0.000218,0.001028,1.046844,0.000010,1.000046
106,"(total_children_(1.0, 2.0])",(product_id_Super Extra Chunky Peanut Butter),0.204682,0.000778,0.000207,0.001009,1.297747,0.000047,1.000232,0.212278,0.000491,0.000218,0.001028,2.093688,0.000114,1.000538
16,"(total_children_(1.0, 2.0])",(product_id_ADJ Rosy Sunglasses),0.204682,0.000778,0.000207,0.001009,1.297747,0.000047,1.000232,0.212278,0.000655,0.000218,0.001028,1.570266,0.000079,1.000374
103,(state_province_CA),(product_id_Big Time Beef TV Dinner),0.144855,0.000662,0.000146,0.001007,1.520030,0.000050,1.000345,0.138499,0.000764,0.000164,0.001182,1.547197,0.000058,1.000419


time: 22.1 ms (started: 2022-01-20 08:22:11 +00:00)


In [13]:
transac_cus_before_dec_group_rules = transac_cus_before_dec_filtered_rules.loc[:,["antecedents", "consequents"]].groupby(["antecedents"]).apply(lambda x:frozenset().union(*x["consequents"].values)).reset_index().rename(columns={"antecedents":"antecedents", 0:"product_set"})
transac_cus_dec_group_rules = transac_cus_dec_filtered_rules.loc[:,["antecedents", "consequents"]].groupby(["antecedents"]).apply(lambda x:frozenset().union(*x["consequents"].values)).reset_index().rename(columns={"antecedents":"antecedents", 0:"product_set"})
diff_rules = pd.merge(transac_cus_before_dec_group_rules, transac_cus_dec_group_rules, on=["antecedents"], how='left', suffixes=('_before', '_dec'))
diff_rules.to_csv('dec_diff_rules.csv')
display(diff_rules)

,antecedents,product_set_before,product_set_dec
0,"(total_children_(3.0, 4.0])","(product_id_Best Choice Salsa Dip, product_id_...","(product_id_Horatio Fondue Mix, product_id_Sun..."
1,"(num_cars_owned_(1.0, 2.0])","(product_id_Nationeel Fudge Brownies, product_...","(product_id_Cutting Edge Chicken Hot Dogs, pro..."
2,"(num_children_at_home_(2.0, 3.0])","(product_id_Horatio Fondue Mix, product_id_Bes...","(product_id_National Egg Substitute, product_i..."
3,(state_province_WA),"(product_id_Fort West Beef Jerky, product_id_C...","(product_id_Kiwi Lox, product_id_Landslide Gra..."
4,"(total_children_(2.0, 3.0])","(product_id_Monarch Rice Medly, product_id_Bet...","(product_id_Nationeel Dried Apricots, product_..."
5,"(num_cars_owned_(2.0, 3.0])","(product_id_Excellent Orange Juice, product_id...","(product_id_Cutting Edge Chicken Hot Dogs, pro..."
6,(state_province_Yucatan),"(product_id_High Top New Potatos, product_id_B...","(product_id_American Sliced Chicken, product_i..."
7,(yearly_income_$10K - $30K),"(product_id_Super Strawberry Jelly, product_id...","(product_id_Horatio Fondue Mix, product_id_Sun..."
8,(occupation_Management),"(product_id_Ebony Lettuce, product_id_Sunset P...","(product_id_National Egg Substitute, product_i..."
9,(education_Bachelors Degree),"(product_id_Atomic White Chocolate Bar, produc...","(product_id_Cutting Edge Chicken Hot Dogs, pro..."


time: 37.9 ms (started: 2022-01-20 08:22:11 +00:00)


In [14]:
display([diff_rules.iloc[18,:][i] for i in range(3)])

[frozenset({'occupation_Skilled Manual'}),
 frozenset({'product_id_ADJ Rosy Sunglasses',
            'product_id_Fabulous Apple Drink',
            'product_id_Fast Fondue Mix',
            'product_id_Nationeel Dried Apples',
            'product_id_Sphinx Wheat Bread',
            'product_id_Steady Whitening Toothpast',
            'product_id_Super Apple Jam',
            'product_id_Super Strawberry Jelly'}),
 frozenset({'product_id_American Chicken Hot Dogs',
            'product_id_American Sliced Ham',
            'product_id_Amigo Scallops',
            'product_id_Atomic Mints',
            'product_id_Atomic Spicy Mints',
            'product_id_BBB Best Brown Sugar',
            'product_id_BBB Best Chunky Peanut Butter',
            'product_id_BBB Best Pepper',
            'product_id_BBB Best Vegetable Oil',
            'product_id_Best Choice Cheese Crackers',
            'product_id_Best Choice Chocolate Donuts',
            'product_id_Best Choice Low Fat Popcorn',
  

time: 2.71 ms (started: 2022-01-20 08:22:11 +00:00)


# Appendix: Find the association rules of transaction date before 12/1 and after 12/1, and compare the two

In [15]:
transaction_before_dec_df = sales_fact_1998_ori_df.copy().drop(['promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_dec_df = sales_fact_dec_1998_ori_df.copy().drop(['promotion_id', 'unit_sales', 'store_sales', 'store_cost'], axis=1) # we only need product_id, time_id, customer_id, store_id
transaction_before_dec_df['product_id'] = transaction_before_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_dec_df['product_id'] = transaction_dec_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0]) # change product_id to product name
transaction_before_dec_df = pd.concat([transaction_before_dec_df, pd.get_dummies(transaction_before_dec_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_dec_df = pd.concat([transaction_dec_df, pd.get_dummies(transaction_dec_df['product_id'])], axis=1).drop(['product_id'],axis=1) # onehot encoding
transaction_before_dec_df = transaction_before_dec_df.drop_duplicates()
transaction_dec_df = transaction_dec_df.drop_duplicates()
transaction_before_dec_df = transaction_before_dec_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
transaction_dec_df = transaction_dec_df.groupby(['time_id', 'customer_id', 'store_id']).sum()
display(transaction_dec_df, transaction_before_dec_df)

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
1066    217         13                          0               0   
        465         13                          0               0   
        484         13                          0               0   
        619         13                          0               0   
        767         4                           0               0   
...                                           ...             ...   
1095    9559        10                          0               0   
        9696        11                          0               0   
        9725        10                          0               0   
        9743        10                          0               0   
        10095       10                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
1066    217         13                                 0   
        465         13                                 0   
        484         13                                 0   
        619         13                                 0   
        767         4                                  0   
...                                                  ...   
1095    9559        10                                 0   
        9696        11                                 0   
        9725        10                                 0   
        9743        10                                 0   
        10095       10                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
1066    217         13                            0   
        465         13                            0   
        484         13                            0   
        619         13                            0   
        767         4                             0   
...                                             ...   
1095    9559        10                            0   
        9696        11                            0   
        9725        10                            0   
        9743        10                            0   
        10095       10                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
1066    217         13                                0                   0   
        465         13                                0                   0   
        484         13                                0                   0   
        619         13                                0                   0   
        767         4                                 0                   0   
...                                                 ...                 ...   
1095    9559        10                                0                   0   
        9696        11                                0                   0   
        9725        10                                0                   0   
        9743        10                                0                   0   
        10095       10                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
1066    217         13                           0   
        465         13                           0   
        484         13                           0   
        619         13                           0   
        767         4                            0   
...                                            ...   
1095    9559        10                           0   
        9696        11                           0   
        9725        10                           0   
        9743 

ADJ Rosy Sunglasses  Akron City Map  \
time_id customer_id store_id                                        
732     534         10                          0               0   
        1013        10                          0               0   
        1325        10                          0               0   
        1425        10                          0               0   
        1550        10                          0               0   
...                                           ...             ...   
1065    9510        16                          0               0   
        9728        16                          0               0   
        9743        10                          0               0   
        9897        16                          0               0   
        10260       11                          0               0   

                              Akron Eyeglass Screwdriver  \
time_id customer_id store_id                               
732     534         10                                 0   
        1013        10                                 0   
        1325        10                                 0   
        1425        10                                 0   
        1550        10                                 0   
...                                                  ...   
1065    9510        16                                 0   
        9728        16                                 0   
        9743        10                                 0   
        9897        16                                 0   
        10260       11                                 0   

                              American Beef Bologna  \
time_id customer_id store_id                          
732     534         10                            0   
        1013        10                            0   
        1325        10                            0   
        1425        10                            0   
        1550        10                            0   
...                                             ...   
1065    9510        16                            0   
        9728        16                            0   
        9743        10                            0   
        9897        16                            0   
        10260       11                            0   

                              American Chicken Hot Dogs  American Cole Slaw  \
time_id customer_id store_id                                                  
732     534         10                                0                   0   
        1013        10                                0                   0   
        1325        10                                0                   0   
        1425        10                                0                   0   
        1550        10                                0                   0   
...                                                 ...                 ...   
1065    9510        16                                0                   0   
        9728        16                                0                   0   
        9743        10                                0                   0   
        9897        16                                0                   0   
        10260       11                                0                   0   

                              American Corned Beef  \
time_id customer_id store_id                         
732     534         10                           0   
        1013        10                           0   
        1325        10                           0   
        1425        10                           0   
        1550        10                           0   
...                                            ...   
1065    9510        16                           0   
        9728        16                           0   
        9743        10                           0   
        9897 

time: 33.6 s (started: 2022-01-20 08:22:11 +00:00)


In [16]:
transac_before_dec_freq_itemset = fpgrowth(transaction_before_dec_df, min_support=0.0001, use_colnames=True) # use fp-growth to find frequent_itemset
transac_before_dec_rules = association_rules(transac_before_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules
transac_dec_freq_itemset = fpgrowth(transaction_dec_df, min_support=0.0003, use_colnames=True) # use fp-growth to find frequent_itemset
transac_dec_rules = association_rules(transac_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules
display(transac_before_dec_rules, transac_dec_rules) 
display(transac_dec_freq_itemset, transac_before_dec_freq_itemset)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Toucan Canned Mixed Fruit),(Big Time Apple Cinnamon Waffles),0.003552,0.003434,0.000205,0.057851,16.846083,0.000193,1.057759
1,(Big Time Apple Cinnamon Waffles),(Toucan Canned Mixed Fruit),0.003434,0.003552,0.000205,0.059829,16.846083,0.000193,1.059859
2,(Big Time Apple Cinnamon Waffles),(High Top Corn on the Cob),0.003434,0.003610,0.000176,0.051282,14.204711,0.000164,1.050249
3,(Bravo Fancy Canned Anchovies),(High Top Oranges),0.003610,0.003434,0.000235,0.065041,18.939615,0.000222,1.065892
4,(High Top Oranges),(Bravo Fancy Canned Anchovies),0.003434,0.003610,0.000235,0.068376,18.939615,0.000222,1.069519
...,...,...,...,...,...,...,...,...,...
6416,"(Imagine Frozen Chicken Thighs, Excellent Diet...","(PigTail Ice Cream Sandwich, Kiwi Lox)",0.000176,0.000117,0.000117,0.666667,5678.333333,0.000117,2.999648
6417,"(Imagine Frozen Chicken Thighs, Kiwi Lox)","(Excellent Diet Soda, PigTail Ice Cream Sandwich)",0.000117,0.000147,0.000117,1.000000,6814.000000,0.000117,inf
6418,"(Excellent Diet Soda, PigTail Ice Cream Sandwich)","(Imagine Frozen Chicken Thighs, Kiwi Lox)",0.000147,0.000117,0.000117,0.800000,6814.000000,0.000117,4.999413
6419,"(PigTail Ice Cream Sandwich, Kiwi Lox)","(Imagine Frozen Chicken Thighs, Excellent Diet...",0.000117,0.000176,0.000117,1.000000,5678.333333,0.000117,inf


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Toucan Canned Peaches),(Cutting Edge Sliced Turkey),0.003438,0.003438,0.000529,0.153846,44.745562,0.000517,1.177755
1,(Cutting Edge Sliced Turkey),(Toucan Canned Peaches),0.003438,0.003438,0.000529,0.153846,44.745562,0.000517,1.177755
2,(Toucan Canned Peaches),(High Top Asparagus),0.003438,0.003967,0.000529,0.153846,38.779487,0.000515,1.177130
3,(High Top Asparagus),(Toucan Canned Peaches),0.003967,0.003438,0.000529,0.133333,38.779487,0.000515,1.149879
4,(Bravo Creamed Corn),(Toucan Canned Peaches),0.003703,0.003438,0.000529,0.142857,41.549451,0.000516,1.162655
...,...,...,...,...,...,...,...,...,...
12955,(Sunset 75 Watt Lightbulb),"(Consolidated Childrens Cold Remedy, Hermanos ...",0.005554,0.000529,0.000529,0.095238,180.047619,0.000526,1.104679
12956,(Washington Orange Juice),"(Consolidated Childrens Cold Remedy, Hermanos ...",0.003967,0.000529,0.000529,0.133333,252.066667,0.000527,1.153236
12957,(Consolidated Childrens Cold Remedy),"(Hermanos Plums, Best Choice Sesame Crackers, ...",0.004496,0.000529,0.000529,0.117647,222.411765,0.000527,1.132734
12958,(Gorilla Low Fat Sour Cream),(Applause Canned Mixed Fruit),0.002645,0.002116,0.000529,0.200000,94.525000,0.000523,1.247355


,support,itemsets
0,0.003438,(Cutting Edge Sliced Turkey)
1,0.003438,(Toucan Canned Peaches)
2,0.001851,(Hilltop Childrens Aspirin)
3,0.001587,(Horatio Low Fat Chips)
4,0.004232,(Blue Label Turkey Noodle Soup)
...,...,...
4530,0.000529,"(Best Choice Sesame Crackers, Hermanos Plums, ..."
4531,0.000529,"(Best Choice Sesame Crackers, Hermanos Plums, ..."
4532,0.000529,"(Consolidated Childrens Cold Remedy, Best Choi..."
4533,0.000529,"(Hermanos Plums, Best Choice Sesame Crackers, ..."


,support,itemsets
0,0.003552,(Toucan Canned Mixed Fruit)
1,0.003434,(Big Time Apple Cinnamon Waffles)
2,0.003434,(High Top Oranges)
3,0.003258,(BBB Best Canola Oil)
4,0.003199,(Excellent Apple Drink)
...,...,...
12515,0.000117,"(PigTail Ice Cream Sandwich, Imagine Ice Cream..."
12516,0.000117,"(Excellent Diet Soda, PigTail Ice Cream Sandwi..."
12517,0.000117,"(Imagine Frozen Chicken Thighs, PigTail Ice Cr..."
12518,0.000117,"(Imagine Frozen Chicken Thighs, PigTail Ice Cr..."


time: 1.54 s (started: 2022-01-20 08:22:45 +00:00)


In [17]:
common_rules = pd.merge(transac_before_dec_rules, transac_dec_rules, on=["antecedents", "consequents"], suffixes=('_before', '_dec'))
common_rules = common_rules.sort_values(by = ['confidence_before', 'confidence_dec', 
                                              'support_before', 'support_dec', 
                                              'lift_before', 'lift_dec', 
                                              'leverage_before', 'leverage_dec', 
                                              'antecedent support_before', 'antecedent support_dec', 
                                              'consequent support_before', 'consequent support_dec', 
                                              'antecedents'], ascending=False)
common_rules.to_csv('dec_trans_common_rules.csv')
display(common_rules)

,antecedents,consequents,antecedent support_before,consequent support_before,support_before,confidence_before,lift_before,leverage_before,conviction_before,antecedent support_dec,consequent support_dec,support_dec,confidence_dec,lift_dec,leverage_dec,conviction_dec
26,(Big Time Popsicles),(Hilltop Silky Smooth Hair Conditioner),0.003170,0.003669,0.000264,0.083333,22.713333,0.000253,1.086907,0.002645,0.004761,0.000529,0.200000,42.011111,0.000516,1.244049
39,(Hermanos Almonds),(Tell Tale Fresh Lima Beans),0.003405,0.003757,0.000264,0.077586,20.651266,0.000251,1.080039,0.004761,0.004496,0.000529,0.111111,24.712418,0.000508,1.119942
13,(Medalist Spaghetti),(Excellent Mango Drink),0.003141,0.003493,0.000235,0.074766,21.405796,0.000224,1.077033,0.002380,0.004761,0.000529,0.222222,46.679012,0.000518,1.279593
27,(Hilltop Silky Smooth Hair Conditioner),(Big Time Popsicles),0.003669,0.003170,0.000264,0.072000,22.713333,0.000253,1.074170,0.004761,0.002645,0.000529,0.111111,42.011111,0.000516,1.122025
5,(Best Choice Frosted Donuts),(BBB Best Apple Butter),0.003287,0.003552,0.000235,0.071429,20.112161,0.000223,1.073098,0.002909,0.002909,0.000529,0.181818,62.495868,0.000520,1.218666
38,(Tell Tale Fresh Lima Beans),(Hermanos Almonds),0.003757,0.003405,0.000264,0.070312,20.651266,0.000251,1.071968,0.004496,0.004761,0.000529,0.117647,24.712418,0.000508,1.127938
14,(Excellent Mango Drink),(Medalist Spaghetti),0.003493,0.003141,0.000235,0.067227,21.405796,0.000224,1.068705,0.004761,0.002380,0.000529,0.111111,46.679012,0.000518,1.122322
32,(Best Choice Frosted Cookies),(Fabulous Strawberry Drink),0.002642,0.003522,0.000176,0.066667,18.927778,0.000167,1.067655,0.003174,0.003967,0.000529,0.166667,42.011111,0.000516,1.195239
6,(BBB Best Apple Butter),(Best Choice Frosted Donuts),0.003552,0.003287,0.000235,0.066116,20.112161,0.000223,1.067276,0.002909,0.002909,0.000529,0.181818,62.495868,0.000520,1.218666
40,(Fast Low Fat Cookies),(Cormorant Paper Plates),0.002700,0.003434,0.000176,0.065217,18.991081,0.000167,1.066094,0.005025,0.003174,0.000529,0.105263,33.166667,0.000513,1.114100


time: 41.8 ms (started: 2022-01-20 08:22:47 +00:00)


In [18]:
dec_only_rules = pd.merge(transac_dec_rules.loc[:, ['antecedents', 'consequents']], transac_before_dec_rules.loc[:, ['antecedents', 'consequents']], on=['antecedents', 'consequents'], how='left', indicator = True).loc[lambda x : x['_merge']!='both']
before_dec_only_rules = pd.merge(transac_dec_rules.loc[:,['antecedents', 'consequents']], transac_before_dec_rules.loc[:, ['antecedents', 'consequents']], on=['antecedents', 'consequents'], how='right', indicator = True).loc[lambda x : x['_merge']!='both']
display(dec_only_rules, before_dec_only_rules)

,antecedents,consequents,_merge
0,(Toucan Canned Peaches),(Cutting Edge Sliced Turkey),left_only
1,(Cutting Edge Sliced Turkey),(Toucan Canned Peaches),left_only
2,(Toucan Canned Peaches),(High Top Asparagus),left_only
3,(High Top Asparagus),(Toucan Canned Peaches),left_only
5,(Toucan Canned Peaches),(Bravo Creamed Corn),left_only
...,...,...,...
12955,(Sunset 75 Watt Lightbulb),"(Consolidated Childrens Cold Remedy, Hermanos ...",left_only
12956,(Washington Orange Juice),"(Consolidated Childrens Cold Remedy, Hermanos ...",left_only
12957,(Consolidated Childrens Cold Remedy),"(Hermanos Plums, Best Choice Sesame Crackers, ...",left_only
12958,(Gorilla Low Fat Sour Cream),(Applause Canned Mixed Fruit),left_only


,antecedents,consequents,_merge
0,(Toucan Canned Mixed Fruit),(Big Time Apple Cinnamon Waffles),right_only
1,(Big Time Apple Cinnamon Waffles),(Toucan Canned Mixed Fruit),right_only
2,(Big Time Apple Cinnamon Waffles),(High Top Corn on the Cob),right_only
3,(Bravo Fancy Canned Anchovies),(High Top Oranges),right_only
4,(High Top Oranges),(Bravo Fancy Canned Anchovies),right_only
...,...,...,...
6416,"(Imagine Frozen Chicken Thighs, Excellent Diet...","(PigTail Ice Cream Sandwich, Kiwi Lox)",right_only
6417,"(Imagine Frozen Chicken Thighs, Kiwi Lox)","(Excellent Diet Soda, PigTail Ice Cream Sandwich)",right_only
6418,"(Excellent Diet Soda, PigTail Ice Cream Sandwich)","(Imagine Frozen Chicken Thighs, Kiwi Lox)",right_only
6419,"(PigTail Ice Cream Sandwich, Kiwi Lox)","(Imagine Frozen Chicken Thighs, Excellent Diet...",right_only


time: 34.3 ms (started: 2022-01-20 08:22:47 +00:00)


In [19]:
transac_dec_rules = association_rules(transac_dec_freq_itemset, metric='confidence', min_threshold=0.05) # find rules

time: 24.5 ms (started: 2022-01-20 08:22:47 +00:00)
